In [1]:
import csv, re, requests
from rdflib import Dataset, URIRef, Literal, Namespace, RDF, RDFS, OWL, XSD
from iribaker import to_iri
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
# no need to query our own dataset
group_numbers = range(1,12) + range(13, 20)
groups = []
for i, el in enumerate(group_numbers):
    groups.append({})
    groups[i]['endpoint'] = 'http://stardog.krw.d2s.labs.vu.nl/group' + str(el)
    groups[i]['vocab'] = 'http://data.krw.d2s.labs.vu.nl/group' + str(el) + '/vocab/'
    groups[i]['data'] = 'http://data.krw.d2s.labs.vu.nl/group' + str(el) + '/resource/'
    groups[i]['number'] = el

In [4]:
def runQuery(endpoint, query='', reasoning=False):
    # Set the endpoint
    myEndpoint = endpoint + '/query'
    sparql = SPARQLWrapper(myEndpoint)
    
    # Write the query
    if query == '':
        query = '''
        SELECT * WHERE {
        ?s ?p ?o .
        } LIMIT 10
        '''
    
    # Set the query
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.addParameter('Accept','application/sparql-results+json')
    sparql.addParameter('reasoning','true' if reasoning else 'false')
    
    # Return the response
    return sparql.query().convert()

In [5]:
AmsterdamURIs = set()
AmsterdamClasses = set()

for group in groups:
    print 'Querying Group ' + str(group['number']) + '...'
    
    ##################################################
    # Start with a (simple) query for Amsterdam URIs using string labels (label, name, naam)
    ##################################################
    print '...Amsterdam query...'

    groupAmsterdamURIs = set()
    
    query = """
    PREFIX krv: <%s>
    PREFIX krr: <%s>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    SELECT DISTINCT ?s ?p ?o WHERE {
        ?s ?p ?o .
    }
    LIMIT %d
    """%(group['vocab'], group['data'], 100000)

    # Run the query and retrieve the response
    response = runQuery(group['endpoint'], query)
    
    # Parse the response
    for el in response['results']['bindings']:
        sub = el['s']['value']
        pred = el['p']['value']
        obj = el['o']['value']
        
        subName = re.split('/|#', sub)[-1].strip().lower()
        predName = re.split('/|#', pred)[-1].strip().lower()
        objName = re.split('/|#', obj)[-1].strip().lower()
        
        if ((predName == 'label' or predName == 'name' or predName == 'naam') and 
            (objName == 'amsterdam')):
            groupAmsterdamURIs.add(sub)
            print sub, pred, obj
    
    ##################################################
    # Check the classes of the Amsterdam URIs
    ##################################################
    print '...Amsterdam classes query...'
    
    for uri in groupAmsterdamURIs:
        isClass = False
        
        query = """
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        SELECT DISTINCT ?type WHERE {
        <%s> a ?type .
        ?type a owl:Class .
        }
        """%(uri)
        
        response = runQuery(group['endpoint'], query, reasoning=True)
        
        for el in response['results']['bindings']:
            if el['type']['value'] == 'http://www.w3.org/2002/07/owl#Nothing': break
            AmsterdamClasses.add(el['type']['value'])
            isClass = True
        
        if isClass:
            AmsterdamURIs.add(uri)
    
    ##################################################
    # More queries...
    ##################################################
    
    print ''

print 'All Amsterdam URIs'
for uri in AmsterdamURIs:
    print uri

print 'All Amsterdam URI Classes'
for uri in AmsterdamClasses:
    print uri
    
print 'Done.'

Querying Group 1...
...Amsterdam query...
http://data.krw.d2s.labs.vu.nl/group1/resource/AMSTERDAM http://www.w3.org/2000/01/rdf-schema#label AMSTERDAM
...Amsterdam classes query...

Querying Group 2...
...Amsterdam query...
...Amsterdam classes query...

Querying Group 3...
...Amsterdam query...
http://data.krw.d2s.labs.vu.nl/group3/resource/Amsterdam http://www.w3.org/2000/01/rdf-schema#label Amsterdam
http://data.krw.d2s.labs.vu.nl/group3/resource/Amsterdam/Amsterdam http://www.w3.org/2000/01/rdf-schema#label Amsterdam
...Amsterdam classes query...

Querying Group 4...
...Amsterdam query...
...Amsterdam classes query...

Querying Group 5...
...Amsterdam query...
http://data.krw.d2s.labs.vu.nl/group5/resource/Amsterdam http://www.w3.org/2000/01/rdf-schema#label Amsterdam
http://data.krw.d2s.labs.vu.nl/group5/resource/Amsterdam http://www.geonames.org/ontology#name Amsterdam
...Amsterdam classes query...

Querying Group 6...
...Amsterdam query...
...Amsterdam classes query...

Queryin